<a href="https://colab.research.google.com/github/mevivekmevivek/Drug-Efficacy-Prediction/blob/main/SWnet_encoded_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

#print(os.listdir("../input"))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SWnet_drug_descriptors.csv to SWnet_drug_descriptors.csv


In [ ]:
import io
#drug_features = pd.read_csv(io.BytesIO(uploaded['complete_desciptors - Copy.csv']))
drug_features = pd.read_csv(io.BytesIO(uploaded['SWnet_drug_descriptors.csv']))

In [ ]:
l11 = []
for x in drug_features.columns:
    
    if drug_features[x].dtype == object:
        l11.append(x)
print(l11)

['Name2', 'Name']


In [ ]:
same_value =[]
new_features=[]
for x in drug_features.columns:
    if drug_features[x].nunique() ==1:
        same_value.append(x)
    else :
        new_features.append(x)

In [ ]:
print(len(new_features))
print(len(same_value))

1555
1203


In [ ]:
Drug_features_columns = drug_features.columns

In [ ]:
# checking for any empty columns
l1=[]
l2 =[]
for x in drug_features.columns:
        if len(drug_features[x].value_counts()) >0:
           l1.append(x)
        else:
           l2.append(x)
print(len(l1))
print(len(l2))


2758
0


In [ ]:
new_Drug_features = drug_features[new_features]

In [ ]:
import numpy as np
new_Drug_features = new_Drug_features.replace([np.inf, -np.inf], np.nan)

In [ ]:
new_Drug_features.isnull().values.any()

False

In [ ]:
new_Drug_features

,Name2,Name,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nC,nN,nO,nS,nF,nCl,nBr,nI,nX,ATS0m,ATS1m,ATS2m,ATS3m,ATS4m,ATS5m,ATS6m,ATS7m,ATS8m,ATS0v,ATS1v,ATS2v,ATS3v,ATS4v,ATS5v,ATS6v,ATS7v,ATS8v,ATS0e,ATS1e,...,PubchemFP758,PubchemFP759,PubchemFP761,PubchemFP763,PubchemFP764,PubchemFP767,PubchemFP770,PubchemFP771,PubchemFP776,PubchemFP777,PubchemFP779,PubchemFP780,PubchemFP782,PubchemFP784,PubchemFP785,PubchemFP788,PubchemFP791,PubchemFP792,PubchemFP793,PubchemFP797,PubchemFP798,PubchemFP799,PubchemFP800,PubchemFP803,PubchemFP805,PubchemFP806,PubchemFP812,PubchemFP813,PubchemFP815,PubchemFP816,PubchemFP818,PubchemFP819,PubchemFP821,PubchemFP822,PubchemFP824,PubchemFP826,PubchemFP827,PubchemFP830,PubchemFP833,PubchemFP834
0,17-AAG,AUTOGEN_20211014173722502_1,1.1663,1.360256,159.6677,92.948099,0,0,85,42,43,31,3,8,0,0,0,0,0,0,7152.210658,7382.798091,10532.312170,13097.548600,14659.146780,16268.237330,16266.432080,16611.845590,17528.229720,16926.69370,21159.90253,30261.90076,38276.93333,44716.25332,50147.60848,52940.01586,53117.88012,53424.02483,660.068584,670.876852,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AEW541,AUTOGEN_20211014173722502_2,-0.9645,0.930260,48.5019,73.158997,21,23,62,33,29,27,5,1,0,0,0,0,0,0,5161.545369,6196.679338,8466.966146,8722.979518,8610.948065,8096.970122,7961.163789,7090.978095,6359.008199,13769.34658,17996.31673,25910.55934,28190.97383,28065.22119,27676.60389,26636.69915,24576.80989,21734.80444,462.789284,513.792392,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,AZD0530,AUTOGEN_20211014173722502_3,-0.8460,0.715716,79.6564,80.547376,16,17,70,38,32,27,5,5,0,0,1,0,0,1,7445.168065,7641.007968,11160.141270,11948.325710,11697.021140,11562.665430,12378.272460,13518.698520,11676.988360,15232.13188,19455.55482,28994.96578,32671.04363,31558.57889,30781.03150,32631.15380,34907.83244,32985.96866,548.427165,594.865638,...,1,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,0,0,0,1
3,AZD6244,AUTOGEN_20211014173722502_4,0.5819,0.338608,48.0315,52.514895,15,16,42,27,15,17,4,3,0,1,1,1,0,3,12022.710260,5969.549145,9189.786465,10627.916770,11766.501210,9133.518475,10190.494580,9859.441852,10506.876100,10672.89595,12549.38130,18339.97830,20307.25859,19504.81920,18335.58454,19439.23925,18350.12092,16406.11277,348.265010,358.311716,...,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1
4,Erlotinib,AUTOGEN_20211014173722502_5,-0.0209,0.000437,47.7828,60.564239,16,17,52,29,23,22,3,4,0,0,0,0,0,0,4809.640285,5279.705423,6944.111040,7645.436695,7370.507577,7555.836294,7796.020363,7492.742953,6709.832431,11627.80359,14158.85737,19270.83509,21473.74710,21385.45214,21740.14954,22289.46982,22484.15691,20522.09402,404.427796,425.950940,...,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
5,Irinotecan,AUTOGEN_20211014173722502_6,-2.8925,8.366556,95.6815,92.630134,16,18,81,43,38,33,4,6,0,0,0,0,0,0,7119.918627,8179.943637,12802.161360,15236.220390,14781.020550,13564.794360,12700.980750,11367.835080,10702.060700,17428.84140,22985.72570,36072.87715,44193.34623,45229.66456,42583.74243,39285.37062,34995.21184,32373.06720,625.055500,675.788068,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,L-685458,AUTOGEN_20211014173722502_7,3.1787,10.104134,197.1093,112.525236,0,0,101,49,52,39,4,6,0,0,0,0,0,0,7999.728249,8504.235259,12209.307390,12890.506350,13915.453100,16380.270540,17060.922480,17855.896200,18491.190160,20405.11523,25831.03341,37248.88040,43345.26061,44358.73930,47508.31698,52521.12300,57396.12656,60454.64040,764.357092,788.907828,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,LBW242,AUTOGEN_20211014173722502_8,-1.9190,3.682561,124.8234,81.529306,0,0,75,33,42,27,4,2,0,0,0,0,0,0,5234.526153,6041.569284,8663.441889,9581.759467,9390.204860,9816.162840,10402.347690,10169.367650,8901.213653

In [ ]:
new_Drug_features.drop(columns = ['Name'], inplace = True)
y = new_Drug_features['Name2']

new_Drug_features.drop(columns = ['Name2'], inplace = True)


In [ ]:
train_scaled = minmax_scale(new_Drug_features, axis = 0)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, y, train_size = 0.9, random_state = seed(2017))

In [ ]:
### Define the encoder dimension
encoding_dim = 50

In [ ]:
from tensorflow.keras.layers import LeakyReLU
ncol = train_scaled.shape[1]
input_dim = Input(shape = (ncol, ))

# Encoder Layers
encoded1 = Dense(2000)(input_dim)
encoded1= LeakyReLU(alpha=0.3)(encoded1)

encoded2 = Dense(1000)(encoded1)
encoded2= LeakyReLU(alpha=0.3)(encoded2)

#encoded3 = Dense(250)(encoded2)
#encoded3= LeakyReLU(alpha=0.3)(encoded3)


encoded4 = Dense(encoding_dim)(encoded2)
encoded4= LeakyReLU(alpha=0.3)(encoded4)


encoder_1 = Model(input_dim, encoded4, name ='encoder_1')

# Decoder Layers
#decoded1 = Dense(250, activation = 'relu')(encoded4)
decoded2 = Dense(1000, activation = 'relu')(encoded4)
decoded3 = Dense(2000, activation = 'relu')(decoded2)
decoded4 = Dense(ncol, activation = 'relu')(decoded3)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded4)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mae')


In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1553)]            0         
_________________________________________________________________
dense (Dense)                (None, 2000)              3108000   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              2001000   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                50050     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 50)                0     

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 2000, batch_size = 15, shuffle = False, validation_data = (X_test, X_test))

Epoch 1/2000
2/2 [==============================] - 1s 264ms/step - loss: 0.4217 - val_loss: 0.3034
Epoch 2/2000
2/2 [==============================] - 0s 114ms/step - loss: 0.3317 - val_loss: 0.2916
Epoch 3/2000
2/2 [==============================] - 0s 114ms/step - loss: 0.2966 - val_loss: 0.2698
Epoch 4/2000
2/2 [==============================] - 0s 106ms/step - loss: 0.2740 - val_loss: 0.3008
Epoch 5/2000
2/2 [==============================] - 0s 108ms/step - loss: 0.2944 - val_loss: 0.2544
Epoch 6/2000
2/2 [==============================] - 0s 122ms/step - loss: 0.2542 - val_loss: 0.2491
Epoch 7/2000
2/2 [==============================] - 0s 114ms/step - loss: 0.2461 - val_loss: 0.2583
Epoch 8/2000
2/2 [==============================] - 0s 120ms/step - loss: 0.2479 - val_loss: 0.2467
Epoch 9/2000
2/2 [==============================] - 0s 112ms/step - loss: 0.2349 - val_loss: 0.2331
Epoch 10/2000
2/2 [==============================] - 0s 111ms/step - loss: 0.2158 - val_loss: 0.2364

In [ ]:
encoded_input = Input(shape = (encoding_dim, ))
encoded_train_1 = pd.DataFrame(encoder_1.predict(train_scaled))
encoded_train_1 = encoded_train_1.add_prefix('drug_feature_')

In [ ]:
encoded_train_1_columns = encoded_train_1.columns
encoded_train_1_columns

Index(['drug_feature_0', 'drug_feature_1', 'drug_feature_2', 'drug_feature_3',
       'drug_feature_4', 'drug_feature_5', 'drug_feature_6', 'drug_feature_7',
       'drug_feature_8', 'drug_feature_9', 'drug_feature_10',
       'drug_feature_11', 'drug_feature_12', 'drug_feature_13',
       'drug_feature_14', 'drug_feature_15', 'drug_feature_16',
       'drug_feature_17', 'drug_feature_18', 'drug_feature_19',
       'drug_feature_20', 'drug_feature_21', 'drug_feature_22',
       'drug_feature_23', 'drug_feature_24', 'drug_feature_25',
       'drug_feature_26', 'drug_feature_27', 'drug_feature_28',
       'drug_feature_29', 'drug_feature_30', 'drug_feature_31',
       'drug_feature_32', 'drug_feature_33', 'drug_feature_34',
       'drug_feature_35', 'drug_feature_36', 'drug_feature_37',
       'drug_feature_38', 'drug_feature_39', 'drug_feature_40',
       'drug_feature_41', 'drug_feature_42', 'drug_feature_43',
       'drug_feature_44', 'drug_feature_45', 'drug_feature_46',
       'drug

In [ ]:
#encoded_train_columns = encoded_train.columns
for l in encoded_train_1_columns:
  if (encoded_train_1[l] == 0).all():
    print("TRUE", l)

In [ ]:
encoded_train_1['Name'] = y
from google.colab import files
encoded_train_1.to_csv('drug_encoded.csv') 
files.download('drug_encoded.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving CCLE_Swnet_rna.csv to CCLE_Swnet_rna.csv


In [ ]:
rna_t = pd.read_csv('CCLE_Swnet_rna.csv')

print (~rna_t.columns.duplicated())
rna_t = rna_t.loc[:, ~rna_t.columns.duplicated()]


[ True  True  True ...  True  True  True]


In [ ]:
rna_t

,Cell_Line,A1CF,ABCC5,ABCF1,ABHD4,ABHD6,ABI1,ABL1,ABL2,ACAA1,ACAT2,ACBD3,ACD,ACKR3,ACLY,ACOT9,ACSL3,ACSL6,ACVR1,ACVR2A,ADAM10,ADAT1,ADGRE5,ADGRG1,ADH5,ADI1,ADO,ADRB2,AFF1,AFF3,AFF4,AGL,AKAP8,AKAP8L,AKAP9,AKR7A2,AKT1,AKT2,AKT3,ALAS1,...,WHSC1L1,WIF1,WIPF2,WNK2,WRN,WT1,WWTR1,XBP1,XPA,XPC,XPNPEP1,XPO1,XPO7,YKT6,YME1L1,YTHDF1,YWHAE,ZBTB16,ZCCHC8,ZDHHC6,ZEB1,ZFHX3,ZFP36,ZMIZ1,ZMYM2,ZMYM3,ZNF131,ZNF274,ZNF318,ZNF331,ZNF384,ZNF395,ZNF429,ZNF451,ZNF479,ZNF521,ZNF586,ZNF589,ZNRF3,ZRSR2
0,22RV1_PROSTATE,-0.078135,-0.047979,-0.007551,-0.078826,-0.093184,-0.008702,-0.035462,-0.082019,0.020303,0.161903,0.013685,-0.010342,-0.102593,0.415464,-0.098766,0.160637,-0.094105,-0.092781,-0.094508,-0.018485,0.058199,-0.074365,-0.095659,0.112842,0.117791,-0.067747,-0.082192,-0.064870,0.007211,-0.051490,-0.057129,-0.039635,-0.000616,0.584688,-0.002688,0.249636,0.178909,-0.101874,0.030978,...,0.006290,-0.094220,-0.067316,-0.067431,-0.054482,-0.098191,-0.090652,0.247565,-0.067517,-0.016903,0.039553,0.297632,0.134365,0.107202,0.123316,-0.013421,1.686755,-0.035606,0.003499,-0.023808,-0.102708,-0.089530,-0.075603,-0.085847,-0.013852,-0.029822,-0.031606,-0.072179,-0.065819,-0.058798,0.065796,-0.044987,-0.057619,-0.051806,-0.102852,-0.102824,-0.072581,-0.069560,-0.084609,-0.083170
1,42MGBA_CENTRAL_NERVOUS_SYSTEM,-0.141742,-0.106508,0.217639,-0.040339,-0.098107,-0.041462,-0.009984,-0.086607,-0.000808,0.077247,-0.021677,0.012278,-0.126835,0.319931,-0.043321,0.083558,-0.141780,-0.016799,-0.104689,0.177991,-0.018812,0.072253,-0.127068,0.338322,-0.008629,-0.106470,-0.133301,-0.102714,-0.139457,0.000315,-0.114523,-0.057453,0.034812,-0.069300,0.013014,0.561338,0.098929,-0.105579,-0.023420,...,-0.042546,-0.141858,-0.097990,-0.141858,-0.115878,-0.133030,-0.014553,0.297746,-0.117311,-0.088543,0.024784,0.133814,-0.034028,0.495982,0.252562,0.067064,3.260213,-0.141664,-0.093886,-0.006887,-0.082155,-0.109606,0.008600,-0.007197,-0.107283,-0.071507,0.040117,-0.060589,-0.119905,-0.115375,-0.001544,-0.086878,-0.126022,-0.098300,-0.141780,-0.093731,-0.129042,-0.130126,-0.130707,-0.113090
2,5637_URINARY_TRACT,-0.136656,-0.103540,0.116922,-0.114223,-0.130835,-0.054733,0.024980,-0.065783,-0.001689,0.057101,-0.079597,-0.055543,0.010503,0.606727,0.153574,0.147275,-0.135956,-0.097757,-0.124758,0.057543,-0.048286,0.015624,0.197077,0.606470,-0.040440,-0.105861,0.100898,-0.116727,-0.133930,-0.064126,-0.120485,-0.066925,-0.023643,-0.063021,0.008993,0.396174,0.171107,-0.058748,0.012971,...,-0.028506,-0.136508,-0.056759,-0.122842,-0.114554,-0.122400,-0.090611,0.036288,-0.100225,-0.108808,-0.006515,0.211995,-0.031047,0.405567,0.033563,0.009656,1.826913,-0.136656,-0.079413,-0.022354,-0.136435,-0.122511,-0.099783,-0.089579,-0.083612,-0.093889,-0.041803,-0.100704,-0.118053,-0.114186,-0.038451,-0.053664,-0.127741,-0.108660,-0.136656,-0.136656,-0.116838,-0.127115,-0.116212,-0.100483
3,639V_URINARY_TRACT,-0.139272,-0.038257,0.185140,-0.107543,-0.111542,-0.057074,-0.064311,-0.102287,0.069347,0.154516,-0.069224,0.040894,-0.138548,0.475500,0.009050,0.044703,-0.138624,-0.119846,-0.131349,0.020477,-0.019250,-0.048046,-0.137824,0.167962,0.150212,-0.096611,-0.137329,-0.104686,-0.139234,-0.026602,-0.117332,-0.047627,0.091858,-0.082289,0.014574,0.467616,0.119473,-0.116685,0.143622,...,-0.041952,-0.139272,-0.090631,-0.136263,-0.106515,-0.122055,-0.034067,0.033542,-0.093221,-0.059054,0.041008,0.374600,-0.032887,0.288745,0.173371,0.016059,5.695731,-0.139272,-0.066444,-0.024469,-0.095659,-0.130435,0.156268,-0.120608,-0.046409,-0.052541,0.126863,-0.083280,-0.073719,-0.094211,0.070452,-0.123426,-0.117675,-0.069110,-0.139272,-0.139081,-0.122512,-0.076195,-0.104800,-0.099773
4,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.114391,-0.047160,0.127130,-0.111325,-0.104127,-0.014949,-0.008429,-0.092379,0.023783,0.015520,-0.038607,-0.035315,-0.114294,0.271727,-0.114004,0.017715,-0.114327,-0.107871,-0.109582,0.042729,-0.036477,-0.053099,-0.109937,0.155178,-0.049258,-0.066945,-0.114294,-0.041189,-0.112971,-0.080275,-0.099157

In [ ]:
# checking for columns with same value across 223 drugs
rna_same_value =[]
rna_new_features=[]
for x in rna_t.columns:
    if rna_t[x].nunique() ==1:
        rna_same_value.append(x)
    else :
        rna_new_features.append(x)

In [ ]:
print(len(rna_same_value))
print(len(rna_new_features))

11
1468


In [ ]:
rna_t = rna_t[rna_new_features]
rna_t.isnull().values.any()


False

In [ ]:
rna_t

,Cell_Line,A1CF,ABCC5,ABCF1,ABHD4,ABHD6,ABI1,ABL1,ABL2,ACAA1,ACAT2,ACBD3,ACD,ACKR3,ACLY,ACOT9,ACSL3,ACSL6,ACVR1,ACVR2A,ADAM10,ADAT1,ADGRE5,ADGRG1,ADH5,ADI1,ADO,ADRB2,AFF1,AFF3,AFF4,AGL,AKAP8,AKAP8L,AKAP9,AKR7A2,AKT1,AKT2,AKT3,ALAS1,...,WHSC1L1,WIF1,WIPF2,WNK2,WRN,WT1,WWTR1,XBP1,XPA,XPC,XPNPEP1,XPO1,XPO7,YKT6,YME1L1,YTHDF1,YWHAE,ZBTB16,ZCCHC8,ZDHHC6,ZEB1,ZFHX3,ZFP36,ZMIZ1,ZMYM2,ZMYM3,ZNF131,ZNF274,ZNF318,ZNF331,ZNF384,ZNF395,ZNF429,ZNF451,ZNF479,ZNF521,ZNF586,ZNF589,ZNRF3,ZRSR2
0,22RV1_PROSTATE,-0.078135,-0.047979,-0.007551,-0.078826,-0.093184,-0.008702,-0.035462,-0.082019,0.020303,0.161903,0.013685,-0.010342,-0.102593,0.415464,-0.098766,0.160637,-0.094105,-0.092781,-0.094508,-0.018485,0.058199,-0.074365,-0.095659,0.112842,0.117791,-0.067747,-0.082192,-0.064870,0.007211,-0.051490,-0.057129,-0.039635,-0.000616,0.584688,-0.002688,0.249636,0.178909,-0.101874,0.030978,...,0.006290,-0.094220,-0.067316,-0.067431,-0.054482,-0.098191,-0.090652,0.247565,-0.067517,-0.016903,0.039553,0.297632,0.134365,0.107202,0.123316,-0.013421,1.686755,-0.035606,0.003499,-0.023808,-0.102708,-0.089530,-0.075603,-0.085847,-0.013852,-0.029822,-0.031606,-0.072179,-0.065819,-0.058798,0.065796,-0.044987,-0.057619,-0.051806,-0.102852,-0.102824,-0.072581,-0.069560,-0.084609,-0.083170
1,42MGBA_CENTRAL_NERVOUS_SYSTEM,-0.141742,-0.106508,0.217639,-0.040339,-0.098107,-0.041462,-0.009984,-0.086607,-0.000808,0.077247,-0.021677,0.012278,-0.126835,0.319931,-0.043321,0.083558,-0.141780,-0.016799,-0.104689,0.177991,-0.018812,0.072253,-0.127068,0.338322,-0.008629,-0.106470,-0.133301,-0.102714,-0.139457,0.000315,-0.114523,-0.057453,0.034812,-0.069300,0.013014,0.561338,0.098929,-0.105579,-0.023420,...,-0.042546,-0.141858,-0.097990,-0.141858,-0.115878,-0.133030,-0.014553,0.297746,-0.117311,-0.088543,0.024784,0.133814,-0.034028,0.495982,0.252562,0.067064,3.260213,-0.141664,-0.093886,-0.006887,-0.082155,-0.109606,0.008600,-0.007197,-0.107283,-0.071507,0.040117,-0.060589,-0.119905,-0.115375,-0.001544,-0.086878,-0.126022,-0.098300,-0.141780,-0.093731,-0.129042,-0.130126,-0.130707,-0.113090
2,5637_URINARY_TRACT,-0.136656,-0.103540,0.116922,-0.114223,-0.130835,-0.054733,0.024980,-0.065783,-0.001689,0.057101,-0.079597,-0.055543,0.010503,0.606727,0.153574,0.147275,-0.135956,-0.097757,-0.124758,0.057543,-0.048286,0.015624,0.197077,0.606470,-0.040440,-0.105861,0.100898,-0.116727,-0.133930,-0.064126,-0.120485,-0.066925,-0.023643,-0.063021,0.008993,0.396174,0.171107,-0.058748,0.012971,...,-0.028506,-0.136508,-0.056759,-0.122842,-0.114554,-0.122400,-0.090611,0.036288,-0.100225,-0.108808,-0.006515,0.211995,-0.031047,0.405567,0.033563,0.009656,1.826913,-0.136656,-0.079413,-0.022354,-0.136435,-0.122511,-0.099783,-0.089579,-0.083612,-0.093889,-0.041803,-0.100704,-0.118053,-0.114186,-0.038451,-0.053664,-0.127741,-0.108660,-0.136656,-0.136656,-0.116838,-0.127115,-0.116212,-0.100483
3,639V_URINARY_TRACT,-0.139272,-0.038257,0.185140,-0.107543,-0.111542,-0.057074,-0.064311,-0.102287,0.069347,0.154516,-0.069224,0.040894,-0.138548,0.475500,0.009050,0.044703,-0.138624,-0.119846,-0.131349,0.020477,-0.019250,-0.048046,-0.137824,0.167962,0.150212,-0.096611,-0.137329,-0.104686,-0.139234,-0.026602,-0.117332,-0.047627,0.091858,-0.082289,0.014574,0.467616,0.119473,-0.116685,0.143622,...,-0.041952,-0.139272,-0.090631,-0.136263,-0.106515,-0.122055,-0.034067,0.033542,-0.093221,-0.059054,0.041008,0.374600,-0.032887,0.288745,0.173371,0.016059,5.695731,-0.139272,-0.066444,-0.024469,-0.095659,-0.130435,0.156268,-0.120608,-0.046409,-0.052541,0.126863,-0.083280,-0.073719,-0.094211,0.070452,-0.123426,-0.117675,-0.069110,-0.139272,-0.139081,-0.122512,-0.076195,-0.104800,-0.099773
4,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.114391,-0.047160,0.127130,-0.111325,-0.104127,-0.014949,-0.008429,-0.092379,0.023783,0.015520,-0.038607,-0.035315,-0.114294,0.271727,-0.114004,0.017715,-0.114327,-0.107871,-0.109582,0.042729,-0.036477,-0.053099,-0.109937,0.155178,-0.049258,-0.066945,-0.114294,-0.041189,-0.112971,-0.080275,-0.099157

In [ ]:
rna_y = rna_t['Cell_Line']
rna_t.drop(columns =['Cell_Line'], inplace = True)
rna_y

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0                              22RV1_PROSTATE
1               42MGBA_CENTRAL_NERVOUS_SYSTEM
2                          5637_URINARY_TRACT
3                          639V_URINARY_TRACT
4      697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE
                        ...                  
464                                 WM88_SKIN
465                               WM983B_SKIN
466               YKG1_CENTRAL_NERVOUS_SYSTEM
467                              ZR751_BREAST
468                             ZR7530_BREAST
Name: Cell_Line, Length: 469, dtype: object

In [ ]:
rna_scaled = minmax_scale(rna_t, axis = 0)
X_train_rna, X_test_rna, Y_train_rna, Y_test_rna = train_test_split(rna_scaled, rna_y, train_size = 0.9, random_state = seed(2017))
mcol = X_train_rna.shape[1]
encoding_dim1 = 100

In [ ]:
print(mcol)
print(X_train_rna.shape)

1467
(422, 1467)


In [ ]:
from tensorflow.keras.layers import LeakyReLU
input_dim1 = Input(shape = (mcol, ))

# Encoder Layers
#encoded1 = Dense(20000)(input_dim1)
#encoded1= LeakyReLU(alpha=0.3)(encoded1)
#encoded2 = Dense(15000, activation = 'relu')(input_dim1)
#encoded3 = Dense(10000, activation = 'relu')(encoded2)
#encoded4 = Dense(5000)(encoded1)
#encoded4= LeakyReLU(alpha=0.3)(encoded4)
#encoded5 = Dense(2000, activation = 'relu')(encoded2)
#encoded6 = Dense(1750, activation = 'relu')(encoded5)
#encoded7 = Dense(1500, activation = 'relu')(encoded6)
#encoded8 = Dense(1250, activation = 'relu')(encoded7)
#encoded9 = Dense(1000)(encoded1)
#encoded9= LeakyReLU(alpha=0.3)(encoded9)
#encoded10 = Dense(750, activation = 'relu')(encoded9)
#encoded11 = Dense(500, activation = 'relu')(encoded10)
#encoded12 = Dense(250, activation = 'relu')(encoded9)
encoded13 = Dense(encoding_dim1)(input_dim1)
encoded13= LeakyReLU(alpha=0.3)(encoded13)

encoder1_rna = Model(inputs = input_dim1, outputs = encoded13)

# Decoder Layers
#decoded1 = Dense(mcol, activation = 'relu')(encoded13)
#decoded2 = Dense(500, activation = 'relu')(decoded1)
#decoded3 = Dense(750, activation = 'relu')(decoded2)
#decoded4 = Dense(1000, activation = 'relu')(encoded13)
#decoded5 = Dense(1250, activation = 'relu')(decoded4)
#decoded6 = Dense(1500, activation = 'relu')(decoded5)
#decoded7 = Dense(1750, activation = 'relu')(decoded6)
#decoded8 = Dense(2000, activation = 'relu')(encoded13)
#decoded9 = Dense(5000, activation = 'relu')(decoded4)
#decoded10 = Dense(10000, activation = 'relu')(decoded9)
#decoded11 = Dense(15000, activation = 'relu')(decoded1)
#decoded12 = Dense(20000, activation = 'relu')(encoded13)
decoded13 = Dense(mcol, activation = 'relu')(encoded13)

# Combine Encoder and Deocder layers
autoencoder1_rna = Model(inputs = input_dim1, outputs = decoded13)

# Compile the Model
autoencoder1_rna.compile(optimizer = 'adam', loss = 'mae')

In [ ]:
autoencoder1_rna.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1467)]            0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               146800    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1467)              148167    
Total params: 294,967
Trainable params: 294,967
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train_rna

array([[0.2571013 , 0.18086554, 0.20858779, ..., 0.30964799, 0.20599903,
        0.18165832],
       [0.38374262, 0.07140611, 0.11091326, ..., 0.4437231 , 0.28288559,
        0.27087046],
       [0.5390678 , 0.30293239, 0.10764899, ..., 0.59332386, 0.40844929,
        0.26659588],
       ...,
       [0.2480008 , 0.10216368, 0.32310585, ..., 0.30959329, 0.20498688,
        0.16511106],
       [0.21512517, 0.11032567, 0.391512  , ..., 0.35403362, 0.19967256,
        0.13558254],
       [0.29986666, 0.09813041, 0.21140328, ..., 0.49539692, 0.21725801,
        0.27503991]])

In [ ]:
autoencoder1_rna.fit(X_train_rna, X_train_rna, epochs = 30, batch_size = 32, shuffle = False, validation_data = (X_test_rna, X_test_rna))

Epoch 1/30
14/14 [==============================] - 1s 15ms/step - loss: 0.1377 - val_loss: 0.1082
Epoch 2/30
14/14 [==============================] - 0s 7ms/step - loss: 0.1021 - val_loss: 0.0935
Epoch 3/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0891 - val_loss: 0.0846
Epoch 4/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0811 - val_loss: 0.0803
Epoch 5/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0764 - val_loss: 0.0767
Epoch 6/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0735 - val_loss: 0.0807
Epoch 7/30
14/14 [==============================] - 0s 5ms/step - loss: 0.0751 - val_loss: 0.0746
Epoch 8/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0724 - val_loss: 0.0751
Epoch 9/30
14/14 [==============================] - 0s 5ms/step - loss: 0.0706 - val_loss: 0.0710
Epoch 10/30
14/14 [==============================] - 0s 6ms/step - loss: 0.0681 - val_loss: 0.0705
Epoch 11/30
14/14 

In [ ]:
encoding_dim1

100

In [ ]:

encoded_input1 = Input(shape = (encoding_dim1, ))

In [ ]:
encoded_train1_rna = pd.DataFrame(encoder1_rna.predict(rna_scaled))
encoded_train1_rna = encoded_train1_rna.add_prefix('rna_feature_')

In [ ]:
encoded_train_columns = encoded_train1_rna.columns
for l in encoded_train_columns:
  if (encoded_train1_rna[l] == 0).all():
    print("TRUE", l)

In [ ]:
encoded_train1_rna

,rna_feature_0,rna_feature_1,rna_feature_2,rna_feature_3,rna_feature_4,rna_feature_5,rna_feature_6,rna_feature_7,rna_feature_8,rna_feature_9,rna_feature_10,rna_feature_11,rna_feature_12,rna_feature_13,rna_feature_14,rna_feature_15,rna_feature_16,rna_feature_17,rna_feature_18,rna_feature_19,rna_feature_20,rna_feature_21,rna_feature_22,rna_feature_23,rna_feature_24,rna_feature_25,rna_feature_26,rna_feature_27,rna_feature_28,rna_feature_29,rna_feature_30,rna_feature_31,rna_feature_32,rna_feature_33,rna_feature_34,rna_feature_35,rna_feature_36,rna_feature_37,rna_feature_38,rna_feature_39,...,rna_feature_60,rna_feature_61,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99
0,-0.301314,0.134451,-0.376868,0.572477,-0.224422,0.267582,0.601426,-0.388899,0.831199,-0.179678,-0.085712,0.432328,-0.364315,0.082179,-0.499525,-0.150441,-0.057271,-0.020424,0.092279,-0.139864,0.382644,0.570077,-0.170550,-0.027130,-0.146516,0.291577,-0.238129,-0.173664,-0.257655,-0.317957,-0.153306,-0.142601,-0.114594,-0.034224,-0.231430,-0.104951,-0.164685,0.673485,0.176826,-0.238549,...,-0.118780,-0.316584,-0.231453,-0.159303,-0.360333,-0.326478,-0.188079,-0.234464,0.541293,-0.151192,-0.357576,0.303813,0.203682,-0.229511,-0.075198,-0.348189,-0.124644,0.273728,0.586258,-0.112521,-0.069596,-0.552787,0.321668,-0.064490,0.215941,-0.082031,-0.046448,-0.270705,-0.518597,0.614536,-0.060595,-0.063233,-0.078384,-0.094329,-0.175151,0.145530,-0.172264,0.248255,-0.276777,-0.257414
1,-0.011573,-0.085532,-0.424264,0.421463,-0.384453,-0.079747,0.399570,-0.114717,0.853058,-0.237078,0.643100,0.661906,-0.405894,-0.027089,-0.376296,-0.193322,-0.005212,-0.151537,-0.063083,-0.153622,0.458482,0.279349,-0.075855,-0.025034,-0.281898,-0.005760,-0.309489,-0.203831,-0.176836,-0.120905,-0.457361,-0.203331,-0.166977,-0.110810,-0.083104,-0.111660,-0.184449,0.545308,0.018463,-0.123385,...,-0.158031,-0.157194,0.088910,0.019039,-0.008279,-0.068812,-0.118034,0.103175,0.515240,-0.041167,-0.226657,-0.072091,0.039059,-0.144125,-0.484094,-0.044728,-0.134534,-0.007787,0.483395,-0.061206,-0.131528,-0.315569,-0.023476,-0.065488,0.154469,-0.115644,0.024269,-0.211308,-0.183053,0.399384,-0.163809,-0.031316,0.356518,-0.080448,-0.099319,0.118954,0.632569,-0.023119,-0.105471,-0.143662
2,-0.004840,-0.030604,-0.400187,0.443474,-0.326154,-0.033309,0.690595,-0.100051,0.974209,-0.194639,0.198645,0.539367,-0.317212,-0.073118,-0.299777,-0.314115,-0.067673,-0.132793,-0.130606,0.418749,0.503520,0.357533,-0.160966,0.076289,0.270371,0.051769,-0.275137,-0.290095,-0.081382,-0.046338,-0.439404,-0.041107,-0.121784,-0.045650,-0.024641,-0.063260,-0.090232,0.416090,-0.048490,0.099893,...,0.103630,-0.211969,-0.093538,-0.023408,-0.078906,-0.206819,-0.051679,-0.164914,0.408331,-0.244048,-0.284817,-0.024718,0.074200,-0.107486,-0.307268,-0.174639,-0.114241,0.107851,0.302003,-0.033901,-0.135274,-0.421179,-0.004912,0.216603,0.209366,-0.196000,0.066582,-0.062639,-0.151172,0.470378,-0.160611,-0.065432,-0.068146,-0.272074,-0.083790,0.108309,0.278595,0.117432,-0.157015,-0.146897
3,-0.014277,-0.013541,-0.325419,0.437591,-0.376245,-0.057270,0.498835,-0.091371,0.722404,-0.068406,0.290654,0.915461,-0.484614,-0.040068,-0.298779,-0.027394,-0.078967,-0.143999,-0.052309,-0.001730,0.201374,0.466724,-0.154038,0.000105,-0.004664,0.155553,-0.363266,-0.141512,-0.260557,-0.066832,-0.395917,0.224474,-0.212091,-0.134235,-0.092176,0.129005,-0.243806,0.317495,0.101189,-0.193713,...,-0.035099,-0.223090,-0.058499,-0.126203,-0.035854,-0.229028,-0.

In [ ]:
from google.colab import files
encoded_train1_rna.to_csv('CCLE_rna.csv') 
files.download('CCLE_rna.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Creating the dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SWNET_CCLE_rna.csv to SWNET_CCLE_rna.csv


In [ ]:
cell_lines = pd.read_csv('CCLE_DRUG_CELL_RESPONSE.csv')
drugs = pd.read_csv('SWnet_drug_encoded.csv')
rna =  pd.read_csv('SWNET_CCLE_rna.csv')

In [ ]:
drugs.head()

,drug_id,drug_feature_0,drug_feature_1,drug_feature_2,drug_feature_3,drug_feature_4,drug_feature_5,drug_feature_6,drug_feature_7,drug_feature_8,drug_feature_9,drug_feature_10,drug_feature_11,drug_feature_12,drug_feature_13,drug_feature_14,drug_feature_15,drug_feature_16,drug_feature_17,drug_feature_18,drug_feature_19,drug_feature_20,drug_feature_21,drug_feature_22,drug_feature_23,drug_feature_24,drug_feature_25,drug_feature_26,drug_feature_27,drug_feature_28,drug_feature_29,drug_feature_30,drug_feature_31,drug_feature_32,drug_feature_33,drug_feature_34,drug_feature_35,drug_feature_36,drug_feature_37,drug_feature_38,drug_feature_39,drug_feature_40,drug_feature_41,drug_feature_42,drug_feature_43,drug_feature_44,drug_feature_45,drug_feature_46,drug_feature_47,drug_feature_48,drug_feature_49,Name
0,17-AAG,-1.956586,-0.474576,-0.780631,6.164061,4.436620,-1.312353,-2.940358,6.136838,-1.529956,-1.822189,-0.938761,-1.483607,-0.092640,-4.380275,-0.531817,-9.708456,-2.160835,1.628740,-1.525341,-0.620277,-0.827656,-0.061147,-0.712054,-0.686311,-0.659081,-0.940660,-0.611098,-1.188937,-4.699968,-0.298720,-0.541766,-1.557857,-0.618236,-3.704009,-0.774503,23.632736,-0.000647,1.670824,-1.914134,-0.438370,-1.526649,-3.070570,10.770574,-5.116450,-2.049668,5.494534,3.807558,-1.492909,-2.712419,-4.321742,17-AAG
1,AEW541,-1.052432,-1.149278,-2.216624,-1.009032,-0.363988,-0.346549,-1.179518,-0.293534,-1.080699,11.749937,-2.182076,-0.649531,-3.091208,-1.105015,-1.161328,-6.968275,-0.827228,-1.669150,-0.911403,-1.446392,0.848104,-0.911550,-1.255707,-1.000124,-1.618658,-1.091895,5.643084,-3.906678,-3.126408,-0.004002,-2.932980,-0.601403,-0.890766,-0.849436,-1.309063,15.182837,-1.503873,-1.209804,-0.739746,-1.043138,-0.742526,-2.019331,-1.062836,-4.882543,-2.807438,4.520480,-0.177591,-0.569415,6.449222,10.559195,AEW541
2,AZD0530,1.810849,-0.663086,-1.716837,-1.377752,0.345021,1.573613,-3.588721,-3.242747,-1.596638,4.272050,-3.934561,-0.441740,-5.235165,-2.834446,-2.007092,-8.078067,-0.867717,-2.786162,-0.113276,-2.125827,4.003398,-1.722171,-0.710631,-0.453443,-0.822305,-3.924650,14.111383,-2.953753,-2.663138,-0.315195,-3.022753,-0.394643,-2.453179,-0.653691,-2.516657,20.939535,6.156994,-0.793594,-1.009562,1.424393,-0.628876,-1.973842,-4.110436,-5.443328,-2.259321,7.227814,-0.943351,0.313364,4.500875,18.085087,AZD0530
3,AZD6244,-1.942905,-2.034560,-0.716308,12.648503,8.316502,-2.452022,-0.303439,3.759297,-1.405490,4.498765,-0.802919,-0.045587,-1.229634,6.451338,-2.486610,-1.152201,1.999026,-1.386697,-0.907508,0.826007,-0.608036,-1.836700,-0.985952,0.986765,-1.374009,-0.909597,27.303165,-3.271579,-4.386701,-0.094435,-2.217811,-1.479310,-0.529518,-0.600775,-0.354560,-0.265366,-2.809811,1.538768,-3.067508,-1.002830,-0.603472,-0.677905,14.550945,-5.057751,-4.138292,13.056404,4.634395,-1.443001,-0.546654,9.045773,AZD6244
4,Erlotinib,-1.966129,1.370342,-3.132047,1.563613,-2.775326,9.497423,-1.593121,-0.331260,-0.196856,1.569406,-0.219634,-0.377034,-3.853178,4.657860,-2.831063,-2.976212,-0.818965,-1.066420,-0.111399,-2.123485,-1.440016,-3.452081,-0.560754,1.914433,1.507829,-2.960958,21.805600,-0.789283,-3.138276,0.277480,-5.065104,-1.544863,-3.814370,-1.174844,-1.694423,2.720667,-0.518327,-1.196903,-2.451179,-0.564193,3.432489,-0.160603,5.968016,-2.823254,-0.902347,9.443917,-1.443483,7.676171,8.434189,20.725464,Erlotinib


In [ ]:
full_data = pd.merge(cell_lines, drugs, on = 'drug_id')

In [ ]:
full_data = pd.merge(full_data,rna, on = 'cell_line_id')

In [ ]:
full_data

,Unnamed: 0,cell_line_id,drug_id,labels,drug_feature_0,drug_feature_1,drug_feature_2,drug_feature_3,drug_feature_4,drug_feature_5,drug_feature_6,drug_feature_7,drug_feature_8,drug_feature_9,drug_feature_10,drug_feature_11,drug_feature_12,drug_feature_13,drug_feature_14,drug_feature_15,drug_feature_16,drug_feature_17,drug_feature_18,drug_feature_19,drug_feature_20,drug_feature_21,drug_feature_22,drug_feature_23,drug_feature_24,drug_feature_25,drug_feature_26,drug_feature_27,drug_feature_28,drug_feature_29,drug_feature_30,drug_feature_31,drug_feature_32,drug_feature_33,drug_feature_34,drug_feature_35,...,rna_feature_60,rna_feature_61,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99
0,1,22RV1_PROSTATE,17-AAG,-1.109567,-1.956586,-0.474576,-0.780631,6.164061,4.436620,-1.312353,-2.940358,6.136838,-1.529956,-1.822189,-0.938761,-1.483607,-0.092640,-4.380275,-0.531817,-9.708456,-2.160835,1.628740,-1.525341,-0.620277,-0.827656,-0.061147,-0.712054,-0.686311,-0.659081,-0.940660,-0.611098,-1.188937,-4.699968,-0.298720,-0.541766,-1.557857,-0.618236,-3.704009,-0.774503,23.632736,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
1,504,22RV1_PROSTATE,AEW541,0.845836,-1.052432,-1.149278,-2.216624,-1.009032,-0.363988,-0.346549,-1.179518,-0.293534,-1.080699,11.749937,-2.182076,-0.649531,-3.091208,-1.105015,-1.161328,-6.968275,-0.827228,-1.669150,-0.911403,-1.446392,0.848104,-0.911550,-1.255707,-1.000124,-1.618658,-1.091895,5.643084,-3.906678,-3.126408,-0.004002,-2.932980,-0.601403,-0.890766,-0.849436,-1.309063,15.182837,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
2,1007,22RV1_PROSTATE,AZD0530,2.079442,1.810849,-0.663086,-1.716837,-1.377752,0.345021,1.573613,-3.588721,-3.242747,-1.596638,4.272050,-3.934561,-0.441740,-5.235165,-2.834446,-2.007092,-8.078067,-0.867717,-2.786162,-0.113276,-2.125827,4.003398,-1.722171,-0.710631,-0.453443,-0.822305,-3.924650,14.111383,-2.953753,-2.663138,-0.315195,-3.022753,-0.394643,-2.453179,-0.653691,-2.516657,20.939535,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
3,1511,22RV1_PROSTATE,AZD6244,2.079442,-1.942905,-2.034560,-0.716308,12.648503,8.316502,-2.452022,-0.303439,3.759297,-1.405490,4.498765,-0.802919,-0.045587,-1.229634,6.451338,-2.486610,-1.152201,1.999026,-1.386697,-0.907508,0.826007,-0.608036,-1.836700,-0.985952,0.986765,-1.374009,-0.909597,27.303165,-3.271579,-4.386701,-0.094435,-2.217811,-1.479310,-0.529518,-0.600775,-0.354560,-0.265366,...,-0.211957,0.428756,-0.306962,0.074894

In [ ]:
full_data

,Unnamed: 0,cell_line_id,drug_id,labels,drug_feature_0,drug_feature_1,drug_feature_2,drug_feature_3,drug_feature_4,drug_feature_5,drug_feature_6,drug_feature_7,drug_feature_8,drug_feature_9,drug_feature_10,drug_feature_11,drug_feature_12,drug_feature_13,drug_feature_14,drug_feature_15,drug_feature_16,drug_feature_17,drug_feature_18,drug_feature_19,drug_feature_20,drug_feature_21,drug_feature_22,drug_feature_23,drug_feature_24,drug_feature_25,drug_feature_26,drug_feature_27,drug_feature_28,drug_feature_29,drug_feature_30,drug_feature_31,drug_feature_32,drug_feature_33,drug_feature_34,drug_feature_35,...,rna_feature_60,rna_feature_61,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99
0,1,22RV1_PROSTATE,17-AAG,-1.109567,-1.956586,-0.474576,-0.780631,6.164061,4.436620,-1.312353,-2.940358,6.136838,-1.529956,-1.822189,-0.938761,-1.483607,-0.092640,-4.380275,-0.531817,-9.708456,-2.160835,1.628740,-1.525341,-0.620277,-0.827656,-0.061147,-0.712054,-0.686311,-0.659081,-0.940660,-0.611098,-1.188937,-4.699968,-0.298720,-0.541766,-1.557857,-0.618236,-3.704009,-0.774503,23.632736,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
1,504,22RV1_PROSTATE,AEW541,0.845836,-1.052432,-1.149278,-2.216624,-1.009032,-0.363988,-0.346549,-1.179518,-0.293534,-1.080699,11.749937,-2.182076,-0.649531,-3.091208,-1.105015,-1.161328,-6.968275,-0.827228,-1.669150,-0.911403,-1.446392,0.848104,-0.911550,-1.255707,-1.000124,-1.618658,-1.091895,5.643084,-3.906678,-3.126408,-0.004002,-2.932980,-0.601403,-0.890766,-0.849436,-1.309063,15.182837,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
2,1007,22RV1_PROSTATE,AZD0530,2.079442,1.810849,-0.663086,-1.716837,-1.377752,0.345021,1.573613,-3.588721,-3.242747,-1.596638,4.272050,-3.934561,-0.441740,-5.235165,-2.834446,-2.007092,-8.078067,-0.867717,-2.786162,-0.113276,-2.125827,4.003398,-1.722171,-0.710631,-0.453443,-0.822305,-3.924650,14.111383,-2.953753,-2.663138,-0.315195,-3.022753,-0.394643,-2.453179,-0.653691,-2.516657,20.939535,...,-0.211957,0.428756,-0.306962,0.074894,-0.104850,-0.079918,-0.026254,0.387534,-0.115765,-0.157008,-0.342113,0.450303,-0.009033,-0.206539,-0.030349,-0.383171,0.229141,-0.315994,0.433992,-0.262697,-0.154096,-0.125621,-0.120546,-0.308676,-0.150721,-0.141939,-0.281012,-0.179641,0.334084,-0.274132,-0.129330,-0.120443,0.434562,-0.190237,-0.096193,-0.241801,0.757970,0.436450,0.340322,0.137018
3,1511,22RV1_PROSTATE,AZD6244,2.079442,-1.942905,-2.034560,-0.716308,12.648503,8.316502,-2.452022,-0.303439,3.759297,-1.405490,4.498765,-0.802919,-0.045587,-1.229634,6.451338,-2.486610,-1.152201,1.999026,-1.386697,-0.907508,0.826007,-0.608036,-1.836700,-0.985952,0.986765,-1.374009,-0.909597,27.303165,-3.271579,-4.386701,-0.094435,-2.217811,-1.479310,-0.529518,-0.600775,-0.354560,-0.265366,...,-0.211957,0.428756,-0.306962,0.074894

In [ ]:
from google.colab import files
full_data.to_csv('CCLE_Integrateddata.csv') 
files.download('CCLE_Integrateddata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler,StandardScaler,OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error,auc

import warnings
warnings.filterwarnings('ignore')

In [ ]:
y = full_data['labels']
full_data.drop(columns =['labels'], inplace = True)

In [ ]:
train_data, test_data, train_ic50, test_ic50 = train_test_split( full_data, y, test_size=1/5, random_state=0)

In [ ]:
train_data.drop(columns=['drug_id', 'cell_line_id', 'Unnamed: 0', 'Name'], inplace= True)
test_data.drop(columns=['drug_id', 'cell_line_id', 'Unnamed: 0', 'Name'], inplace= True)

In [ ]:
!pip install xgboost

In [ ]:
import xgboost
from xgboost import plot_importance
from collections import OrderedDict
lasso = Lasso(alpha =0.0005)
model = xgboost.XGBRegressor(tree_method='gpu_hist',colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=9,
                 min_child_weight=1.5,
                 n_estimators=2000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42
                 ) 



In [ ]:
def mse(y, y_pred):
    return mean_squared_error(y, y_pred)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_validate

n_folds = 5 # number of folds
def get_cv_scores(model, X, y, print_scores=True):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X) # create folds
    scores = cross_validate(model, X, y, cv=kf,
                         scoring=('r2', 'neg_mean_squared_error'),
                         return_train_score=True)
      
    #rmse = -cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = kf) # get rmse
    #r2 = cross_val_score(model, X, y, scoring="r2", cv = kf) # get rmse
    if print_scores:
      print(scores['test_neg_mean_squared_error'], "Mean MSE :",scores['test_neg_mean_squared_error'].mean())
      print(scores['test_r2'], "Mean R2 :",scores['test_r2'].mean())
    #     print(f'Root mean squared error: {rmse.mean():.3f} ({rmse.std():.3f})')
    #     print(f'R2: {r2.mean():.3f} ({r2.std():.3f})')
    return scores

In [ ]:
get_cv_scores(model,train_data, train_ic50)

[15:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:22:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[nan nan nan nan nan] Mean MSE : nan
[nan nan nan nan nan] Mean R2 : nan


{'fit_time': array([0.06386733, 0.02613997, 0.03143406, 0.02540255, 0.02672625]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_neg_mean_squared_error': array([nan, nan, nan, nan, nan]),
 'test_r2': array([nan, nan, nan, nan, nan]),
 'train_neg_mean_squared_error': array([nan, nan, nan, nan, nan]),
 'train_r2': array([nan, nan, nan, nan, nan])}

In [ ]:
model.fit(train_data, train_ic50)

[15:23:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBoostError: ignored

In [ ]:
train_data

,drug_feature_0,drug_feature_1,drug_feature_2,drug_feature_3,drug_feature_4,drug_feature_5,drug_feature_6,drug_feature_7,drug_feature_8,drug_feature_9,drug_feature_10,drug_feature_11,drug_feature_12,drug_feature_13,drug_feature_14,drug_feature_15,drug_feature_16,drug_feature_17,drug_feature_18,drug_feature_19,drug_feature_20,drug_feature_21,drug_feature_22,drug_feature_23,drug_feature_24,drug_feature_25,drug_feature_26,drug_feature_27,drug_feature_28,drug_feature_29,drug_feature_30,drug_feature_31,drug_feature_32,drug_feature_33,drug_feature_34,drug_feature_35,drug_feature_36,drug_feature_37,drug_feature_38,drug_feature_39,...,rna_feature_60,rna_feature_61,rna_feature_62,rna_feature_63,rna_feature_64,rna_feature_65,rna_feature_66,rna_feature_67,rna_feature_68,rna_feature_69,rna_feature_70,rna_feature_71,rna_feature_72,rna_feature_73,rna_feature_74,rna_feature_75,rna_feature_76,rna_feature_77,rna_feature_78,rna_feature_79,rna_feature_80,rna_feature_81,rna_feature_82,rna_feature_83,rna_feature_84,rna_feature_85,rna_feature_86,rna_feature_87,rna_feature_88,rna_feature_89,rna_feature_90,rna_feature_91,rna_feature_92,rna_feature_93,rna_feature_94,rna_feature_95,rna_feature_96,rna_feature_97,rna_feature_98,rna_feature_99
10547,1.056450,-1.306880,-0.934255,-0.830061,0.191106,-3.137583,-0.192908,8.959335,-0.682367,9.805870,-2.390277,-0.523926,-1.494401,2.548598,-0.748282,-5.085078,-0.652876,-0.217645,-1.375085,-0.517128,-0.051226,-1.002550,-1.052970,-0.953865,-0.496943,-0.458967,8.811615,-5.130061,-3.985179,-0.968832,-1.820672,-0.696853,-1.258274,-0.357422,-0.496957,16.383825,-0.682188,-0.818469,-0.507147,-1.706619,...,-0.057266,0.872567,-0.344557,0.400233,-0.157590,-0.169936,-0.073493,-0.043246,-0.043233,-0.131607,-0.302335,0.410101,0.212705,-0.259570,-0.022132,-0.177059,0.436935,-0.427181,0.379842,-0.184570,-0.056595,-0.248386,0.019746,-0.249938,-0.349767,-0.064505,-0.154971,-0.106906,0.798335,-0.459342,-0.108331,-0.139691,0.691577,-0.287679,-0.245786,-0.383247,0.938820,-0.080074,0.586398,-0.047631
3806,-0.516650,-0.673703,-0.486068,-2.568626,6.188991,-0.124660,0.158571,-1.714801,-0.377630,5.685633,-0.668121,1.622354,-1.448134,0.576805,-0.465201,-5.131899,-0.137150,-1.346206,-0.129433,-0.335064,-0.139642,-0.288589,-0.405128,0.719856,-1.336045,-0.352909,11.056929,-3.917626,-4.471154,-0.423308,-1.106218,-0.372177,-2.803233,-0.175192,-0.218655,7.495190,-3.052970,-0.939714,-1.271199,-0.261119,...,-0.114755,0.376695,-0.265063,0.056130,-0.129231,-0.244629,-0.142676,0.026498,-0.041474,-0.146765,-0.348190,0.503648,-0.053078,-0.199791,-0.219163,-0.109670,0.111253,-0.123009,0.472147,-0.156291,-0.256008,-0.222603,-0.243451,-0.268725,-0.024608,-0.190399,-0.117668,-0.099040,0.123666,-0.275471,0.015476,-0.038709,0.386626,-0.256056,-0.060220,-0.111007,0.706378,0.006164,0.158844,-0.027214
9485,-3.835104,-2.574440,-3.059989,-0.928543,-0.456411,-0.749268,-1.866032,20.668837,-0.305439,11.156096,-0.976469,-2.178290,-1.819976,-3.798771,-3.130017,-4.081908,-0.319256,-1.166401,0.005865,-1.417186,-1.545988,8.537315,-0.490559,-0.389362,1.443850,1.981405,20.982218,-4.303159,-4.666787,-0.982389,-3.991637,-1.702705,1.498079,-1.264783,-1.733151,5.892119,-5.850943,-0.410739,9.193628,-0.996140,...,-0.114573,0.733738,-0.338747,0.494276,-0.196246,-0.225069,-0.121408,-0.061656,-0.009840,-0.068167,-0.355969,0.485170,0.172667,-0.362699,-0.043463,-0.115461,0.336839,-0.373460,0.485693,-0.138348,-0.121045,-0.222193,-0.150202,-0.272499,-0.296917,-0.090133,-0.070580,-0.141006,0.715828,-0.539746,-0.089906,0.108820,0.655866,-0.306903,-0.194278,-0.264675,0.876340,-0.111490,0.437742,-0.059619
9831,-4.929890,-1.417016,-4.429511,7.164239,-1.756073,4.993905,-0.585915,-0.401162,3.602750,11.489220,-1.649346,-3.119791,-0.753932,-0.337560,-2.172265,-3.629692,-0.530509,-1.266839,-2.310594,-1.441897,-2.539609,-1.491291,-0.407546,-0.756112,-1.770943,-1.256346,-0.159066,-1.480105,-0.656302,0.223670,-2.940376,-1.649859,-0.739218,-2.250231,-0.794338,-0.519304,-4.072651,-1.449000,-2.121141,